<a href="https://colab.research.google.com/github/tu-mo/hoc_may/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Khai báo thư viện**

In [1]:
import torch
from torch.utils.data import DataLoader
from torch import nn

import torchvision
from torchvision import transforms

import matplotlib.pyplot as plt

from collections import namedtuple

from sklearn.metrics import classification_report

# **Lấy dữ liệu từ github**

In [2]:
!git clone https://github.com/tu-mo/hoc_may.git

Cloning into 'hoc_may'...
remote: Enumerating objects: 8326, done.
remote: Counting objects: 100% (8326/8326), done.
remote: Compressing objects: 100% (8325/8325), done.
remote: Total 8326 (delta 1), reused 8326 (delta 1), pack-reused 0
Receiving objects: 100% (8326/8326), 488.02 MiB | 29.43 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Checking out files: 100% (8326/8326), done.


# **Chuẩn bị dữ liệu**








In [21]:
TrainTest = namedtuple('TrainTest', ['train', 'test'])

def get_classes():
  classes = ['2C', '3C', '4C']
  return classes

def prepare_data(size):
  transform_train = transforms.Compose([
    transforms.Resize((size,size)),
    transforms.ToTensor()
  ])
  transform_test = transforms.Compose([
    transforms.Resize((size,size)), 
    transforms.ToTensor()
  ])
  trainset = torchvision.datasets.ImageFolder(root='/content/hoc_may/train', transform=transform_train)
  testset = torchvision.datasets.ImageFolder(root='/content/hoc_may/test', transform=transform_test)
  return TrainTest(train=trainset, test=testset)

def prepare_loader(datasets):
  trainloader = DataLoader(dataset=datasets.train, batch_size=32, shuffle=True, num_workers=4)
  testloader = DataLoader(dataset=datasets.test, batch_size=32, shuffle=False, num_workers=4)
  return TrainTest(train=trainloader, test=testloader)


# **Train + Test**

In [13]:
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  model.train()
  running_loss = 0.0
  reporting_steps = 32
  for i, (images, labels) in enumerate(loader):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    loss = loss_func(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if i % reporting_steps == reporting_steps-1:
      print(f"Epoch {epoch} step {i} ave_loss {running_loss/reporting_steps:.4f}")
      running_loss = 0.0

def test_epoch(epoch, model, loader, device):
  ytrue = []
  ypred = []
  with torch.no_grad():
    model.eval()
    for i, (images, labels) in enumerate(loader):
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)
      ytrue += list(labels.cpu().numpy())
      ypred += list(predicted.cpu().numpy())

  return ypred, ytrue

# **Thực thi**

In [26]:
def main(model = 'vgg16', size = 32):
  classes = get_classes()
  datasets = prepare_data(size)
  loaders = prepare_loader(datasets)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  if model == 'vgg16':
    print("vgg16")
    model = torchvision.models.vgg16()
    # vì bài toán có 3 phân lớp
    model.classifier[-1] = torch.nn.Linear(in_features=4096, out_features=3)
  elif model == 'vgg19':
    print("vgg19")
    model = torchvision.models.vgg19()
    # vì bài toán có 3 phân lớp
    model.classifier[-1] = torch.nn.Linear(in_features=4096, out_features=3)
  # elif model == 'googlenet':
  #   model = torchvision.models.googlenet()
  #   model.fc = torch.nn.Linear(in_features=1024, out_features=3)
  elif model == 'resnet50':
    print("resnet50")
    model = torchvision.models.resnet50()
    model.fc = torch.nn.Linear(in_features=2048, out_features=3)

  model.to(device)
  loss_func = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
  for epoch in range(10):
    train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    ypred, ytrue = test_epoch(epoch, model, loaders.test, device)
    print(classification_report(ytrue, ypred, target_names=classes))

In [ ]:
main('vgg16',32)

In [ ]:
main('vgg16',224)

In [ ]:
main('vgg19',32)

In [ ]:
main('vgg19',32)

In [ ]:
main('resnet50',32)

In [ ]:
main('resnet50',32)